In [1]:
import sys
import os
import numpy as np

from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier 

In [2]:
sys.path.append("D:\Study\Python\Sleep-Analysis-with-accelerometer")

import prepare_data

In [4]:
sorce_path = os.path.join("D:\Study\Python\Sleep-Analysis-with-accelerometer", 
                          "ICHI14_dataset\data")

data_path = os.path.join("D:\Study\Python\Sleep-Analysis-with-accelerometer",
                         "statistic_features", "stat_features_standardized.csv")

patient_list = ['002','003','005','007','08a','08b','09a','09b', '10a','011','013','014','15a','15b','016',
            '017','018','019','020','021','022','023','025','026','027','028','029','030','031','032',
            '033','034','035','036','037','038','040','042','043','044','045','047','048','049','051']

statistics_list = ["std_x", "std_y", "std_z"]

#### Saving the statistic features.
#### As scaler=True, firstly we standardize accelerometer data for all dataset, and then calculate statistic features for windows.

In [6]:
#prepare_data.save_statistic_features(patient_list, sorce_path=sorce_path, save_path=data_path, 
#                            window_len=60, n_sleep_stages=1, scaler=True)

In [7]:
kf = KFold(n_splits=5, random_state=5, shuffle=True) # Define the split - into 5 folds #5
kf.get_n_splits(patient_list) 

5

In [8]:
for train_index, test_index in kf.split(patient_list):
    #train_patient_list = [patient_list[i] for i in train_index]
    test_patient_list = [patient_list[i] for i in test_index]
    
    print(test_patient_list)

['007', '08a', '09a', '025', '028', '029', '031', '044', '048']
['002', '005', '08b', '021', '026', '027', '032', '034', '049']
['003', '013', '014', '15b', '020', '022', '035', '036', '045']
['09b', '15a', '017', '019', '023', '037', '042', '047', '051']
['10a', '011', '016', '018', '030', '033', '038', '040', '043']


In [72]:
n_others_windows = 30

## 1. Logistic regression
### 1.1 Statistic features per window: STD for each of 3 axis

In [82]:
statistics_list_std = ["std_x", "std_y", "std_z"]

In [83]:
%%time
accuracy_list = []
f1_score_list = []
for train_index, test_index in kf.split(patient_list):
    train_patient_list = [patient_list[i] for i in train_index]
    test_patient_list = [patient_list[i] for i in test_index]
    
    X_train, y_train = prepare_data.load_stat_features_others_windows(train_patient_list,
                                                                      data_path=data_path,
                                                                      n_others_windows=n_others_windows,
                                                                      statistics_list=statistics_list_std)
    X_test, y_test = prepare_data.load_stat_features_others_windows(test_patient_list,
                                                                    data_path=data_path, 
                                                                    n_others_windows=n_others_windows,
                                                                    statistics_list=statistics_list_std)
    
    
    weights = np.ones(y_train.shape)
    weights[y_train==1] = 1.8

    model1 = LogisticRegression()
    model1.fit(X_train, y_train, sample_weight=weights)
    
    print("\nTrain set result: ")
    y_predict = model1.predict(X_train)
    accuracy_train = metrics.accuracy_score(y_train, y_predict)
    f1_train = metrics.f1_score(y_train, y_predict)
    print("Accuracy on train set: ", accuracy_train)
    print("F1-score on train set: ", f1_train)
    
    print("\nTest set result: ")
    y_predict = model1.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_predict)
    f1_test = metrics.f1_score(y_test, y_predict)
    accuracy_list.append(accuracy)
    f1_score_list.append(f1_test)
    print("Accuracy on test set: ", accuracy)
    print("F1-score on test set: ", f1_test)
    
    print(metrics.classification_report(y_test, y_predict, target_names=["sleep", "awake"]))
    print("Confussion matrix: \n", metrics.confusion_matrix(y_test, y_predict))
    print("\n-------------------------------------------------------")

print("\nMean accuracy =", np.mean(accuracy_list))    
print("\nMean f1-score =", np.mean(f1_score_list))    


Train set result: 
Accuracy on train set:  0.762588702810105
F1-score on train set:  0.7149672846237731

Test set result: 
Accuracy on test set:  0.6735849056603773
F1-score on test set:  0.6183121897407611
             precision    recall  f1-score   support

      sleep       0.65      0.80      0.71      2178
      awake       0.72      0.54      0.62      2062

avg / total       0.68      0.67      0.67      4240

Confussion matrix: 
 [[1735  443]
 [ 941 1121]]

-------------------------------------------------------

Train set result: 
Accuracy on train set:  0.742850459091175
F1-score on train set:  0.6796357012750456

Test set result: 
Accuracy on test set:  0.7460050462573591
F1-score on test set:  0.7449324324324326
             precision    recall  f1-score   support

      sleep       0.72      0.78      0.75      2298
      awake       0.77      0.72      0.74      2458

avg / total       0.75      0.75      0.75      4756

Confussion matrix: 
 [[1784  514]
 [ 694 1764]]



#### 1.1 Results:

weight for awake class = 1:
    
    30 windows: acc =  0.7487, f1-score = 0.6532
    
The problem is that we detect more sleep class than awake. So, the algorithm little overestimates the sleep. One of the reasons may be that we have little more windows for sleep class.

weight for awake class = 1.5:
    
    30 windows: acc = 0.7415, f1-score = 0.6794
    
weight for awake class = 1.7:
    
    30 windows: acc = 0.7342, f1-score = 0.6836

weight for awake class = 1.8:
    
    30 windows: acc = 0.7296, f1-score = 0.6845
    
    32 windows: acc = 0.7307, f1-score = 0.6840
    
weight for awake class = 1.9:
    
    30 windows: acc = 0.7221, f1-score = 0.6826

weight for awake class = 2:
    
    30 windows: acc = 0.7147, f1-score = 0.6810
    
To conclude, it seems reasonable to use weight for awake class = 1.8, as we have best f1-score.

#### 1.1.2 Results for only axis X:

weight for awake class = 1:
    
    32 windows: acc =  0.7263, f1-score = 0.6219
    
weight for awake class = 1.5:
    
    32 windows: acc = 0.7150, f1-score = 0.6514
    
weight for awake class = 1.7:
    
    32 windows: acc = 0.7082, f1-score = 0.6602

weight for awake class = 1.8:
    
    30 windows: acc = 0.7038, f1-score = 0.6651
    
    32 windows: acc = 0.7047, f1-score = 0.6642
    
weight for awake class = 1.9:
    
    32 windows: acc = 0.6976, f1-score = 0.6648

### 1.2 Statistic features per window: PTP for each of 3 axis (peak to peak)

In [27]:
statistics_list_ptp = ["ptp_x", "ptp_y", "ptp_z"]

In [34]:
%%time
accuracy_list = []
f1_score_list = []
for train_index, test_index in kf.split(patient_list):
    train_patient_list = [patient_list[i] for i in train_index]
    test_patient_list = [patient_list[i] for i in test_index]
    
    X_train, y_train = prepare_data.load_stat_features_others_windows(train_patient_list,
                                                                      data_path=data_path,
                                                                      n_others_windows=n_others_windows,
                                                                      statistics_list=statistics_list_ptp)
    X_test, y_test = prepare_data.load_stat_features_others_windows(test_patient_list,
                                                                    data_path=data_path, 
                                                                    n_others_windows=n_others_windows,
                                                                    statistics_list=statistics_list_ptp)
    
    
    weights = np.ones(y_train.shape)
    weights[y_train==1] = 1.8

    model1 = LogisticRegression()
    model1.fit(X_train, y_train, sample_weight=weights)
    
    print("\nTrain set result: ")
    y_predict = model1.predict(X_train)
    accuracy_train = metrics.accuracy_score(y_train, y_predict)
    f1_train = metrics.f1_score(y_train, y_predict)
    print("Accuracy on train set: ", accuracy_train)
    print("F1-score on train set: ", f1_train)
    
    print("\nTest set result: ")
    y_predict = model1.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_predict)
    f1_test = metrics.f1_score(y_test, y_predict)
    accuracy_list.append(accuracy)
    f1_score_list.append(f1_test)
    print("Accuracy on test set: ", accuracy)
    print("F1-score on test set: ", f1_test)
    
    print(metrics.classification_report(y_test, y_predict, target_names=["sleep", "awake"]))
    print("Confussion matrix: \n", metrics.confusion_matrix(y_test, y_predict))
    print("\n-------------------------------------------------------")

print("\nMean accuracy =", np.mean(accuracy_list))    
print("\nMean f1-score =", np.mean(f1_score_list))    


Train set result: 
Accuracy on train set:  0.763894408174851
F1-score on train set:  0.7163222154014051

Test set result: 
Accuracy on test set:  0.6712264150943397
F1-score on test set:  0.6129927817878956
             precision    recall  f1-score   support

      sleep       0.65      0.80      0.71      2178
      awake       0.72      0.54      0.61      2062

avg / total       0.68      0.67      0.66      4240

Confussion matrix: 
 [[1742  436]
 [ 958 1104]]

-------------------------------------------------------

Train set result: 
Accuracy on train set:  0.7502193110708228
F1-score on train set:  0.6879976623566366

Test set result: 
Accuracy on test set:  0.7106812447434819
F1-score on test set:  0.7129745515227368
             precision    recall  f1-score   support

      sleep       0.69      0.73      0.71      2298
      awake       0.73      0.70      0.71      2458

avg / total       0.71      0.71      0.71      4756

Confussion matrix: 
 [[1671  627]
 [ 749 1709]]


#### 1.2 Results:

weight for awake class = 1:
    
    30 windows: acc = 0.7442, f1-score = 0.6516

weight for awake class = 1.5:
    
    30 windows: acc = 0.7376, f1-score = 0.6767
    
weight for awake class = 1.7:
    
    30 windows: acc = 0.7317, f1-score = 0.6814

weight for awake class = 1.8:
    
    30 windows: acc = 0.7285, f1-score = 0.6830

weight for awake class = 1.9:
    
    30 windows: acc = 0.7234, f1-score = 0.6833

weight for awake class = 2:
    
    30 windows: acc = 0.7178, f1-score = 0.6831
    
To conclude, it seems reasonable to use weight for awake class = 1.8, as we have good f1-score and accuracy.

## 2. Gradient Boosting Classifier
### 2.1 Statistic features per window: STD for each of 3 axis

In [ ]:
statistics_list_std = ["std_x"]

In [53]:
%%time
accuracy_list = []
f1_score_list = []
for train_index, test_index in kf.split(patient_list):
    train_patient_list = [patient_list[i] for i in train_index]
    test_patient_list = [patient_list[i] for i in test_index]
    
    X_train, y_train = prepare_data.load_stat_features_others_windows(train_patient_list,
                                                                      data_path=data_path,
                                                                      n_others_windows=n_others_windows)
    X_test, y_test = prepare_data.load_stat_features_others_windows(test_patient_list,
                                                                    data_path=data_path, 
                                                                    n_others_windows=n_others_windows)
    
    
    weights = np.ones(y_train.shape)
    weights[y_train==1] = 1.5

    model1 = GradientBoostingClassifier(n_estimators=40, max_depth=4)
    model1.fit(X_train, y_train, sample_weight=weights)
    
    print("\nTrain set result: ")
    y_predict = model1.predict(X_train)
    accuracy_train = metrics.accuracy_score(y_train, y_predict)
    f1_train = metrics.f1_score(y_train, y_predict)
    print("Accuracy on train set: ", accuracy_train)
    print("F1-score on train set: ", f1_train)
    
    print("\nTest set result: ")
    y_predict = model1.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_predict)
    f1_test = metrics.f1_score(y_test, y_predict)
    accuracy_list.append(accuracy)
    f1_score_list.append(f1_test)
    print("Accuracy on test set: ", accuracy)
    print("F1-score on test set: ", f1_test)
    
    print(metrics.classification_report(y_test, y_predict, target_names=["sleep", "awake"]))
    print("Confussion matrix: \n", metrics.confusion_matrix(y_test, y_predict))
    print("\n-------------------------------------------------------")

print("\nMean accuracy =", np.mean(accuracy_list))    
print("\nMean f1-score =", np.mean(f1_score_list))    


Train set result: 
Accuracy on train set:  0.795004257734885
F1-score on train set:  0.7451838261237739

Test set result: 
Accuracy on test set:  0.6695754716981132
F1-score on test set:  0.6193969030154849
             precision    recall  f1-score   support

      sleep       0.65      0.78      0.71      2178
      awake       0.70      0.55      0.62      2062

avg / total       0.68      0.67      0.66      4240

Confussion matrix: 
 [[1699  479]
 [ 922 1140]]

-------------------------------------------------------

Train set result: 
Accuracy on train set:  0.7815076905082169
F1-score on train set:  0.7161525604011547

Test set result: 
Accuracy on test set:  0.70878889823381
F1-score on test set:  0.6976642654442262
             precision    recall  f1-score   support

      sleep       0.67      0.77      0.72      2298
      awake       0.75      0.65      0.70      2458

avg / total       0.71      0.71      0.71      4756

Confussion matrix: 
 [[1773  525]
 [ 860 1598]]

-

#### 2.1 Results:
##### Parameters: n_estimators = 40, max_depth = 4

weight for awake class = 1:
    
    30 windows: acc = 0.7399, f1-score = 0.6500
    
weight for awake class = 1.2:
    
    30 windows: acc = 0.7368, f1-score = 0.6616
    
weight for awake class = 1.3:
    
    30 windows: acc = 0.7332, f1-score = 0.6652

weight for awake class = 1.5:
    
    30 windows: acc = 0.7255, f1-score = 0.6711
    
weight for awake class = 1.7:
    
    30 windows: acc = 0.7115, f1-score = 0.6685

weight for awake class = 1.8:
    
    30 windows: acc = 0.7092, f1-score = 0.6731  - best

weight for awake class = 1.9:
    
    30 windows: acc = 0.7022, f1-score = 0.6727
     
##### Parameters: n_estimators = 20, max_depth = 4

weight for awake class = 1.2:
    
    30 windows: acc = 0.7338, f1-score = 0.6530

weight for awake class = 1.5:
    
    30 windows: acc = 0.7206, f1-score = 0.6648
    
weight for awake class = 1.7:
    
    30 windows: acc = 0.7065, f1-score = 0.6681

weight for awake class = 1.8:
    
    30 windows: acc = 0.7019, f1-score = 0.6724